In [9]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, Subset, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.OxfordIIITPet(root='./data', split="trainval", download=True, transform=transform)

class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

np.random.seed(42)
all_classes = np.array(list(class_to_idx.keys()))
np.random.shuffle(all_classes)
first_20_classes = all_classes[:20]
first_20_list = [cls.lower() for cls in first_20_classes]

selected_indices = [i for i, (img, label) in enumerate(dataset)
                    if idx_to_class[label].lower() in first_20_list]
print(f"Total images for selected 20 classes: {len(selected_indices)}")

subset_20 = Subset(dataset, selected_indices)

train_size = int(0.8 * len(subset_20))
test_size = len(subset_20) - train_size
train_subset, test_subset = random_split(subset_20, [train_size, test_size])
print(f"Train size: {train_size}, Test size: {test_size}")

def remap_labels(subset_obj):
    images = []
    labels = []
    for img, orig_label in subset_obj:
        class_name = idx_to_class[orig_label].lower()
        if class_name not in first_20_list:
            print("Warning: unexpected class encountered:", class_name)
            continue
        new_label = first_20_list.index(class_name)
        images.append(img)
        labels.append(new_label)
    return torch.stack(images), torch.tensor(labels)

train_images, train_labels = remap_labels(train_subset)
test_images, test_labels = remap_labels(test_subset)

print(f"Remapped train dataset size: {len(train_labels)}")
print(f"Remapped test dataset size: {len(test_labels)}")

train_loader = DataLoader(TensorDataset(train_images, train_labels), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(test_images, test_labels), batch_size=32, shuffle=False)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 20)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")


torch.save(model.state_dict(), "resnet20.pth")
print("Training completed and model saved as resnet20.pth")


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Final Test Accuracy on 20-class dataset: {test_accuracy:.2f}%")


Total images for selected 20 classes: 1988
Train size: 1590, Test size: 398
Remapped train dataset size: 1590
Remapped test dataset size: 398
Epoch [1/10], Loss: 56.4932, Accuracy: 63.71%
Epoch [2/10], Loss: 27.3758, Accuracy: 82.45%
Epoch [3/10], Loss: 17.8102, Accuracy: 88.87%
Epoch [4/10], Loss: 10.1316, Accuracy: 93.77%
Epoch [5/10], Loss: 7.8524, Accuracy: 94.97%
Epoch [6/10], Loss: 9.3905, Accuracy: 94.53%
Epoch [7/10], Loss: 8.0063, Accuracy: 94.59%
Epoch [8/10], Loss: 7.1338, Accuracy: 95.85%
Epoch [9/10], Loss: 5.2838, Accuracy: 97.30%
Epoch [10/10], Loss: 7.0224, Accuracy: 95.53%
Training completed and model saved as resnet20.pth
Final Test Accuracy on 20-class dataset: 60.80%


**Part B**

In [11]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, Subset, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.OxfordIIITPet(root='./data', split="trainval", download=True, transform=transform)

class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

np.random.seed(42)
all_classes = np.array(list(class_to_idx.keys()))
np.random.shuffle(all_classes)
first_20_classes = all_classes[:20]  # Old classes
new_17_classes = all_classes[20:]  # New classes

first_20_list = [cls.lower() for cls in first_20_classes]
new_17_list = [cls.lower() for cls in new_17_classes]

selected_indices_17 = [i for i, (img, label) in enumerate(dataset)
                        if idx_to_class[label].lower() in new_17_list]
print(f"Total images for selected 17 classes: {len(selected_indices_17)}")

subset_17 = Subset(dataset, selected_indices_17)

train_size_17 = int(0.8 * len(subset_17))
test_size_17 = len(subset_17) - train_size_17
train_subset_17, test_subset_17 = random_split(subset_17, [train_size_17, test_size_17])

def remap_labels(subset_obj, class_list):
    images, labels = [], []
    for img, orig_label in subset_obj:
        class_name = idx_to_class[orig_label].lower()
        if class_name not in class_list:
            print("Warning: unexpected class encountered:", class_name)
            continue
        new_label = class_list.index(class_name)
        images.append(img)
        labels.append(new_label)
    return torch.stack(images), torch.tensor(labels)

train_images_17, train_labels_17 = remap_labels(train_subset_17, new_17_list)
test_images_17, test_labels_17 = remap_labels(test_subset_17, new_17_list)

print(f"Remapped train dataset size (17 classes): {len(train_labels_17)}")
print(f"Remapped test dataset size (17 classes): {len(test_labels_17)}")

train_loader_17 = DataLoader(TensorDataset(train_images_17, train_labels_17), batch_size=32, shuffle=True)
test_loader_17 = DataLoader(TensorDataset(test_images_17, test_labels_17), batch_size=32, shuffle=False)

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 20)
model.load_state_dict(torch.load("resnet20.pth"))
model.to(device)

# Modified FC layer for 37 classes
model.fc = nn.Linear(model.fc.in_features, 37).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader_17:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")


torch.save(model.state_dict(), "resnet37.pth")
print("Finetuning Completed. Model saved as resnet37.pth")


test_loader_37 = DataLoader(dataset, batch_size=32, shuffle=False)

selected_indices_20 = [i for i, (img, label) in enumerate(dataset)
                       if idx_to_class[label].lower() in first_20_list]
test_subset_20 = Subset(dataset, selected_indices_20)

test_images_20, test_labels_20 = remap_labels(test_subset_20, first_20_list)
test_loader_20 = DataLoader(TensorDataset(test_images_20, test_labels_20), batch_size=32, shuffle=False)

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

accuracy_37 = evaluate_model(model, test_loader_37)
accuracy_20 = evaluate_model(model, test_loader_20)
accuracy_17 = evaluate_model(model, test_loader_17)

print(f"Final Test Accuracy on Full 37-class dataset: {accuracy_37:.2f}%")
print(f"Accuracy on Original 20-class dataset: {accuracy_20:.2f}%")
print(f"Accuracy on New 17-class dataset: {accuracy_17:.2f}%")


Total images for selected 17 classes: 1692
Remapped train dataset size (17 classes): 1353
Remapped test dataset size (17 classes): 339
Epoch [1/10], Loss: 60.8314, Accuracy: 55.06%
Epoch [2/10], Loss: 20.2399, Accuracy: 85.59%
Epoch [3/10], Loss: 13.3732, Accuracy: 90.17%
Epoch [4/10], Loss: 8.9782, Accuracy: 92.98%
Epoch [5/10], Loss: 7.4219, Accuracy: 94.53%
Epoch [6/10], Loss: 6.2730, Accuracy: 96.16%
Epoch [7/10], Loss: 7.7627, Accuracy: 94.53%
Epoch [8/10], Loss: 5.0917, Accuracy: 96.01%
Epoch [9/10], Loss: 4.2363, Accuracy: 96.90%
Epoch [10/10], Loss: 3.9222, Accuracy: 97.12%
Finetuning Completed. Model saved as resnet37.pth
Final Test Accuracy on Full 37-class dataset: 3.40%
Accuracy on Original 20-class dataset: 4.38%
Accuracy on New 17-class dataset: 70.21%


**Part C**

In [13]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, Subset, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.OxfordIIITPet(root='./data', split="trainval", download=True, transform=transform)

class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

np.random.seed(42)
all_classes = np.array(list(class_to_idx.keys()))
np.random.shuffle(all_classes)
first_20_classes = all_classes[:20]
new_17_classes = all_classes[20:]

first_20_list = [cls.lower() for cls in first_20_classes]
new_17_list = [cls.lower() for cls in new_17_classes]

selected_indices_20 = [i for i, (img, label) in enumerate(dataset)
                        if idx_to_class[label].lower() in first_20_list]
selected_indices_17 = [i for i, (img, label) in enumerate(dataset)
                        if idx_to_class[label].lower() in new_17_list]

old_subset_size = int(0.1 * len(selected_indices_20))
np.random.shuffle(selected_indices_20)
selected_indices_20_small = selected_indices_20[:old_subset_size]

subset_17 = Subset(dataset, selected_indices_17)
subset_20_small = Subset(dataset, selected_indices_20_small)

train_size_17 = int(0.8 * len(subset_17))
test_size_17 = len(subset_17) - train_size_17
train_subset_17, test_subset_17 = random_split(subset_17, [train_size_17, test_size_17])

def remap_labels(subset_obj, class_list):
    images, labels = [], []
    for img, orig_label in subset_obj:
        class_name = idx_to_class[orig_label].lower()
        if class_name not in class_list:
            continue
        new_label = class_list.index(class_name)
        images.append(img)
        labels.append(new_label)
    return torch.stack(images), torch.tensor(labels)

train_images_17, train_labels_17 = remap_labels(train_subset_17, new_17_list)
test_images_17, test_labels_17 = remap_labels(test_subset_17, new_17_list)

train_images_20_small, train_labels_20_small = remap_labels(subset_20_small, first_20_list)

train_images_combined = torch.cat([train_images_20_small, train_images_17])
train_labels_combined = torch.cat([train_labels_20_small, train_labels_17])

train_loader = DataLoader(TensorDataset(train_images_combined, train_labels_combined), batch_size=32, shuffle=True)
test_loader_17 = DataLoader(TensorDataset(test_images_17, test_labels_17), batch_size=32, shuffle=False)

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 20)
model.load_state_dict(torch.load("resnet20.pth"))
model.to(device)

model.fc = nn.Linear(model.fc.in_features, 37).to(device)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), "resnet37_replay.pth")
print("Training Completed with Replay. Model saved as resnet37_replay.pth")

test_loader_37 = DataLoader(dataset, batch_size=32, shuffle=False)

test_subset_20 = Subset(dataset, selected_indices_20)
test_images_20, test_labels_20 = remap_labels(test_subset_20, first_20_list)
test_loader_20 = DataLoader(TensorDataset(test_images_20, test_labels_20), batch_size=32, shuffle=False)

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

accuracy_37 = evaluate_model(model, test_loader_37)
accuracy_20 = evaluate_model(model, test_loader_20)
accuracy_17 = evaluate_model(model, test_loader_17)

print(f"Final Test Accuracy on Full 37-class dataset: {accuracy_37:.2f}%")
print(f"Accuracy on Original 20-class dataset: {accuracy_20:.2f}%")
print(f"Accuracy on New 17-class dataset: {accuracy_17:.2f}%")


Epoch [1/10], Loss: 105.8541, Accuracy: 33.66%
Epoch [2/10], Loss: 74.6432, Accuracy: 49.97%
Epoch [3/10], Loss: 69.3605, Accuracy: 52.80%
Epoch [4/10], Loss: 65.6420, Accuracy: 54.87%
Epoch [5/10], Loss: 62.2031, Accuracy: 58.67%
Epoch [6/10], Loss: 61.8955, Accuracy: 58.16%
Epoch [7/10], Loss: 61.3004, Accuracy: 58.80%
Epoch [8/10], Loss: 57.9077, Accuracy: 60.86%
Epoch [9/10], Loss: 57.9202, Accuracy: 61.83%
Epoch [10/10], Loss: 57.3594, Accuracy: 62.41%
Training Completed with Replay. Model saved as resnet37_replay.pth
Final Test Accuracy on Full 37-class dataset: 5.30%
Accuracy on Original 20-class dataset: 55.73%
Accuracy on New 17-class dataset: 58.41%
